In [1]:
import pandas as pd
df_items = pd.read_csv('./items.csv')
df_reviews = pd.read_csv('./reviews_sentiment_analysis.csv')
df_genres = pd.read_csv('./games_genres.csv')
df_games = pd.read_csv('./games.csv', parse_dates=['release_date'])
df_specs= pd.read_csv('./games_specs.csv')
# Combinar los DataFrames
# Eliminar columnas innecesarias
# df.drop(['release_date'], axis=1, inplace=True)

# Rellenar valores faltantes con ceros
#df.fillna(0, inplace=True)

In [11]:
df_games.head()

,app_name,release_date,id,precios
0,Lost Summoner Kitty,2018-01-04,761140,4.99
1,Ironbound,2018-01-04,643980,0.00
2,Real Pool 3D - Poolians,2017-07-24,670290,0.00
3,弹炸人2222,2017-12-07,767400,0.99
4,Battle Royale Trainer,2018-01-04,772540,3.99


In [2]:
df_gs = df_genres.merge(df_specs, on='item_id', how='inner')
df = df_games.merge(df_gs, on='item_id', how='inner')

df.head()

,app_name,release_date,item_id,precios,Indie,Action,Casual,Adventure,Strategy,Simulation,...,Massively Multiplayer,Racing,Single-player,Multi-player,Full controller support,Partial Controller Support,Steam Leaderboards,Co-op,Cross-Platform Multiplayer,Online Multi-Player
0,Lost Summoner Kitty,2018-01-04,761140,4.99,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Ironbound,2018-01-04,643980,0.00,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
2,Real Pool 3D - Poolians,2017-07-24,670290,0.00,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,弹炸人2222,2017-12-07,767400,0.99,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Battle Royale Trainer,2018-01-04,772540,3.99,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30091 entries, 0 to 30090
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   app_name                    30090 non-null  object        
 1   release_date                30091 non-null  datetime64[ns]
 2   item_id                     30091 non-null  int64         
 3   precios                     30091 non-null  float64       
 4   Indie                       30091 non-null  float64       
 5   Action                      30091 non-null  float64       
 6   Casual                      30091 non-null  float64       
 7   Adventure                   30091 non-null  float64       
 8   Strategy                    30091 non-null  float64       
 9   Simulation                  30091 non-null  float64       
 10  RPG                         30091 non-null  float64       
 11  Free to Play                30091 non-null  float64   

In [19]:
from sklearn.metrics.pairwise import cosine_similarity

def recomendacion_juego(id_producto, num_recomendaciones=5):
    df_gs = df_genres.merge(df_specs, on='item_id', how='inner')
    df = df_games.merge(df_gs, on='item_id', how='inner')
    
    # Obtener el vector de géneros del juego de entrada
    juego_vector = df[df['item_id'] == id_producto].iloc[:, 3:].values.reshape(1, -1)
    # Calcular la similitud del coseno entre el juego de entrada y todos los demás juegos
    similarity_scores = cosine_similarity(df.iloc[:, 3:], juego_vector)
    # Obtener los índices de los juegos más similares
    similar_indices = similarity_scores.argsort(axis=0)[::-1][:num_recomendaciones]

    # Obtener los nombres de los juegos recomendados
    recomendaciones = df.iloc[similar_indices.ravel(), :]['app_name'].values.tolist()

    return recomendaciones

# Ejemplo de uso
id_producto = 70  # ID de un juego de entrada
recomendaciones = recomendacion_juego(id_producto)
print(recomendaciones)

['Unreal Gold', 'Half-Life', 'Descent: FreeSpace – The Great War', 'Freespace 2', 'Descent']
